In [2]:
import librosa
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import random
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torchmetrics
import os
import warnings
import optuna

warnings.filterwarnings('ignore')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

class Config:
    SR = 32000
    N_MFCC = 13
    ROOT_FOLDER = './'
    N_CLASSES = 2
    BATCH_SIZE = 96
    N_EPOCHS = 5
    LR = 3e-4
    SEED = 42
    
CONFIG = Config()

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CONFIG.SEED)

df = pd.read_csv(os.path.join(CONFIG.ROOT_FOLDER, 'train.csv'))
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, random_state=CONFIG.SEED)

def get_mfcc_feature(df, train_mode=True):
    features = []
    labels = []
    for _, row in tqdm(df.iterrows()):
        y, sr = librosa.load(os.path.join(CONFIG.ROOT_FOLDER, row['path']), sr=CONFIG.SR)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CONFIG.N_MFCC)
        mfcc = np.mean(mfcc.T, axis=0)
        features.append(mfcc)

        if train_mode:
            label = row['label']
            label_vector = np.zeros(CONFIG.N_CLASSES, dtype=float)
            label_vector[0 if label == 'fake' else 1] = 1
            labels.append(label_vector)

    if train_mode:
        return features, labels
    return features

train_mfcc, train_labels = get_mfcc_feature(train, True)
val_mfcc, val_labels = get_mfcc_feature(val, True)

class CustomDataset(Dataset):
    def __init__(self, mfcc, label):
        self.mfcc = mfcc
        self.label = label

    def __len__(self):
        return len(self.mfcc)

    def __getitem__(self, index):
        if self.label is not None:
            return self.mfcc[index], self.label[index]
        return self.mfcc[index]

train_dataset = CustomDataset(train_mfcc, train_labels)
val_dataset = CustomDataset(val_mfcc, val_labels)

train_loader = DataLoader(train_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False)

class AudioCNN(nn.Module):
    def __init__(self, input_dim, n_classes, n_layers, hidden_dim):
        super(AudioCNN, self).__init__()
        self.conv_layers = nn.ModuleList()
        self.conv_layers.append(nn.Conv1d(1, hidden_dim, kernel_size=3, stride=1, padding=1))
        for _ in range(n_layers - 1):
            self.conv_layers.append(nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, stride=1, padding=1))
        self.fc = nn.Linear(hidden_dim, n_classes)
        
    def forward(self, x):
        x = x.unsqueeze(1)
        for conv in self.conv_layers:
            x = F.relu(conv(x))
        x = F.adaptive_avg_pool1d(x, 1).squeeze(-1)
        x = self.fc(x)
        return torch.sigmoid(x)

from sklearn.metrics import roc_auc_score

def train(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    best_val_score = 0
    best_model = None
    
    for epoch in range(1, CONFIG.N_EPOCHS+1):
        model.train()
        train_loss = []
        for features, labels in tqdm(iter(train_loader)):
            features = features.float().to(device)
            labels = labels.float().to(device)
            
            optimizer.zero_grad()
            
            output = model(features)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val AUC : [{_val_score:.5f}]')
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
    
    return best_model

def multiLabel_AUC(y_true, y_scores):
    auc_scores = []
    for i in range(y_true.shape[1]):
        auc = roc_auc_score(y_true[:, i], y_scores[:, i])
        auc_scores.append(auc)
    mean_auc_score = np.mean(auc_scores)
    return mean_auc_score
    
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss, all_labels, all_probs = [], [], []
    
    with torch.no_grad():
        for features, labels in tqdm(iter(val_loader)):
            features = features.float().to(device)
            labels = labels.float().to(device)
            
            probs = model(features)
            
            loss = criterion(probs, labels)

            val_loss.append(loss.item())

            all_labels.append(labels.cpu().numpy())
            all_probs.append(probs.cpu().numpy())
        
        _val_loss = np.mean(val_loss)

        all_labels = np.concatenate(all_labels, axis=0)
        all_probs = np.concatenate(all_probs, axis=0)
        
        auc_score = multiLabel_AUC(all_labels, all_probs)
    
    return _val_loss, auc_score

def objective(trial):
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    n_layers = trial.suggest_int('n_layers', 1, 5)
    hidden_dim = trial.suggest_int('hidden_dim', 32, 256)
    
    model = AudioCNN(input_dim=CONFIG.N_MFCC, n_classes=CONFIG.N_CLASSES, n_layers=n_layers, hidden_dim=hidden_dim)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    best_model = train(model, optimizer, train_loader, val_loader, device)
    _, val_score = validation(best_model, nn.BCELoss().to(device), val_loader, device)
    
    return val_score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

best_lr = trial.params['lr']
best_n_layers = trial.params['n_layers']
best_hidden_dim = trial.params['hidden_dim']

final_model = AudioCNN(input_dim=CONFIG.N_MFCC, n_classes=CONFIG.N_CLASSES, n_layers=best_n_layers, hidden_dim=best_hidden_dim)
final_optimizer = torch.optim.Adam(final_model.parameters(), lr=best_lr)

infer_model = train(final_model, final_optimizer, train_loader, val_loader, device)

def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for features in tqdm(iter(test_loader)):
            features = features.float().to(device)
            
            probs = model(features)

            probs  = probs.cpu().detach().numpy()
            predictions += probs.tolist()
    return predictions

test = pd.read_csv(os.path.join(CONFIG.ROOT_FOLDER, 'test.csv'))
test_mfcc = get_mfcc_feature(test, False)
test_dataset = CustomDataset(test_mfcc, None)
test_loader = DataLoader(test_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False)

preds = inference(infer_model, test_loader, device)

submit = pd.read_csv(os.path.join(CONFIG.ROOT_FOLDER, 'sample_submission.csv'))
submit.iloc[:, 1:] = preds
submit.head()

submit.to_csv('./sg2.csv', index=False)


44350it [08:48, 83.90it/s] 
11088it [02:03, 89.86it/s] 
[I 2024-07-01 12:01:44,036] A new study created in memory with name: no-name-f72a080d-8ce8-451f-af88-cc06ff301a2a
100%|██████████| 116/116 [00:01<00:00, 96.65it/s]


Epoch [1], Train Loss : [0.50115] Val Loss : [0.37067] Val AUC : [0.92759]


100%|██████████| 116/116 [00:01<00:00, 105.88it/s]


Epoch [2], Train Loss : [0.31706] Val Loss : [0.28515] Val AUC : [0.95457]


100%|██████████| 116/116 [00:01<00:00, 110.80it/s]


...

100%|██████████| 116/116 [00:01<00:00, 74.39it/s]


Epoch [4], Train Loss : [0.12165] Val Loss : [0.13309] Val AUC : [0.99062]


100%|██████████| 116/116 [00:01<00:00, 75.18it/s]


Epoch [5], Train Loss : [0.10810] Val Loss : [0.11353] Val AUC : [0.99200]


50000it [12:44, 65.37it/s]
100%|██████████| 521/521 [00:06<00:00, 77.43it/s]
